<h1>WordEmbedding<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#作业成果" data-toc-modified-id="作业成果-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>作业成果</a></span><ul class="toc-item"><li><span><a href="#sample.py输出结果" data-toc-modified-id="sample.py输出结果-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>sample.py输出结果</a></span></li><li><span><a href="#输出结果分析与认识" data-toc-modified-id="输出结果分析与认识-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>输出结果分析与认识</a></span></li></ul></li><li><span><a href="#RNN理解与心得体会" data-toc-modified-id="RNN理解与心得体会-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>RNN理解与心得体会</a></span><ul class="toc-item"><li><span><a href="#基础RNN" data-toc-modified-id="基础RNN-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>基础RNN</a></span></li><li><span><a href="#LSTM" data-toc-modified-id="LSTM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LSTM</a></span></li><li><span><a href="#RNN代码分析" data-toc-modified-id="RNN代码分析-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>RNN代码分析</a></span></li></ul></li></ul></div>

# 作业成果

 ## sample.py输出结果
 
```sh
INFO:tensorflow:Restoring parameters from ./output/model.ckpt-53835
2018-05-16 16:26:39,995 - INFO - tf_logging.py:82 - Restoring parameters from ./output/model.ckpt-53835
2018-05-16 16:26:40,029 - DEBUG - sample.py:53 - restore from [./output/model.ckpt-53835]
2018-05-16 16:26:40,670 - DEBUG - sample.py:88 - ==============[江神子]==============
2018-05-16 16:26:40,671 - DEBUG - sample.py:89 - 江神子





刘UNK

水调歌头（和韵）

一里春风，一番春月，一番春色。一里春风，一番一里，一番春月。

一枝春色，一番春去，一番
2018-05-16 16:26:41,129 - DEBUG - sample.py:88 - ==============[蝶恋花]==============
2018-05-16 16:26:41,130 - DEBUG - sample.py:89 - 蝶恋花一点春风，一点春风。





刘UNK

水调歌头（和韵）

一里春风，一番春月，一番春色。一里春风，一番一里，一点春风。

一枝
2018-05-16 16:26:41,584 - DEBUG - sample.py:88 - ==============[渔家傲]==============
2018-05-16 16:26:41,585 - DEBUG - sample.py:89 - 渔家傲





刘UNK

水调歌头（和韵）

一里春风，一番春月，一番春色。一里春风，一番一里，一番春月。

一枝春色，一番春去，一番
```

## 输出结果分析与认识

学会了词牌名, 还学会了姓氏

# RNN理解与心得体会

## 基础RNN

循环神经网络(Recurrent Neural Network. RNN )是一种特殊的前向传播神经网络,它对于序列化数据如语音和文本等具有很强的建模能力, 广泛的用于自然语言处理中的语音识别以及机器翻译等领域.    
标准的RNN由3层构成:输入层、循环层和输出层.   
它的特别之处在于模型的循环层. 顾名思义, 循环, 也就是在循环层加入了自连接和互连接, 通过使用一个短序列, 对所有索引进行逐个的扫描来实现短时记忆的功能. 
简单的基于RBM的模型都是层间节点全连接, 层内节点无连接.    
由于单词之间不是独立的, 都是有语义关系的, 而RNN可以充分的利用上下文信息, 记忆当前单词并时时更新这个时刻之前的所有单词的信息, 基于序列从而更好的预测.    

## LSTM

由于RNN也有梯度消失的问题，因此很难处理长序列的数据, 而RNN的改进算法LSTM(Long Short-Term Memory)可以避免常规RNN的梯度消失的问题.   
RNN的改进之处主要的两大点, 一个是细胞状态, 另一个就是门控结构.   
细胞状态是除了隐藏状态$h^{(t)}$之外的另一隐藏状态.   
门控结构有三个: 遗忘门, 输入门, 输出门  
sigmoid激活的输出为(0,1), 可近似看做是概率, 1为保留, 0为遗忘  
遗忘门输出$f^{(t)}$由隐藏状态$h^{(t-1)}$和本次序列数据$x^{(t)}$，通过sigmoid激活得到   
输入门由隐藏状态$h^{(t-1)}$和本次序列数据$x^{(t)}$，通过sigmoid激活以及tanh激活, 两个激活结果Hadamard乘积得到   
细胞状态的输出由上一细胞状态$C^{(t-1)}$和遗忘门输出$f^{(t)}$的乘积, 加上输入门的结果共同得到.   
输出门隐藏状态$h^{(t-1)}$和本次序列数据$x^{(t)}$，通过sigmoid激活, 并与本层经过tanh激活过的细胞状态Hadamard乘积得到

## RNN代码分析

```py
with tf.variable_scope('rnn'):
    # 创建lstm_cell
    lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(self.dim_embedding,forget_bias=0.0,state_is_tuple=True)
    # 对lstm_cell进行Dropout
    lstm_cell=tf.nn.rnn_cell.DropoutWrapper(lstm_cell,output_keep_prob=self.keep_prob)
    # 重复生成多个lstm_cell并合并为真正的cell
    cell=tf.nn.rnn_cell.MultiRNNCell([lstm_cell]*self.rnn_layers,state_is_tuple=True)
    # 创建初始的state, 并全部用0填充
    self.state_tensor=cell.zero_state(self.batch_size,dtype=tf.float32)
    # 激活一个RNN训练
    seq_output,final_state=tf.nn.dynamic_rnn(cell,data,initial_state=self.state_tensor,scope='rnnlm')
    # 记录最后的state
    self.outputs_state_tensor=final_state
    #----------------------------------------
#flattenit#3x32x128->96x128
seq_output_final=tf.reshape(seq_output,[-1,self.dim_embedding])

with tf.variable_scope('softmax'):
    softmax_w=tf.get_variable("softmax_w",[self.dim_embedding,self.num_words],dtype=tf.float32)
    softmax_b=tf.get_variable("softmax_b",[self.num_words],dtype=tf.float32)
    #96x128->96x5000
    logits=tf.matmul(seq_output_final,softmax_w)+softmax_b
    #----------------------------------------------------------------------
tf.summary.histogram('logits',logits)
```